In [17]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
my_url = 'https://www.hltv.org/results?startDate=2018-09-03&endDate=2019-03-03&stars=1'

page = requests.get(my_url, headers=headers)
page_html = page.text

soup = BeautifulSoup(page_html,"html.parser")

In [18]:
num_matches = int(soup.find_all('span', class_ = ["pagination-data"])[0].text.split(" ")[-2])

In [19]:
all_matches = []

In [20]:
for i in range(0, num_matches, 100):
    my_url = f'https://www.hltv.org/results?offset={i}&startDate=2018-09-03&endDate=2019-03-03&stars=1'

    page = requests.get(my_url, headers=headers)
    page_html = page.text

    soup = BeautifulSoup(page_html,"html.parser")
    matches = soup.find_all('a', class_ = ["a-reset"])
    for match in matches:
        match_link = match["href"]
        if (match_link not in all_matches and match_link[0] == '/'):
            all_matches.append(match_link)
        

In [21]:
all_matches

['/matches/2331080/mibr-vs-astralis-iem-katowice-2019',
 '/matches/2331079/natus-vincere-vs-ence-iem-katowice-2019',
 '/matches/2331078/astralis-vs-nip-iem-katowice-2019',
 '/matches/2331077/mibr-vs-renegades-iem-katowice-2019',
 '/matches/2331076/liquid-vs-ence-iem-katowice-2019',
 '/matches/2331271/valiance-vs-ldlc-esea-mdl-season-30-europe',
 '/matches/2331075/natus-vincere-vs-faze-iem-katowice-2019',
 '/matches/2331318/avangar-vs-uruguay-lootbet-season-1',
 '/matches/2331313/pro100-vs-avangar-lootbet-season-1',
 '/matches/2331095/ago-vs-valiance-esea-mdl-season-30-europe',
 '/matches/2331312/winstrike-vs-avangar-lootbet-season-1',
 '/matches/2331263/izako-boars-vs-valiance-esea-mdl-season-30-europe',
 '/matches/2331288/bojestvata-vs-avangar-noxfire-league-season-2',
 '/matches/2331260/valiance-vs-virtuspro-esea-mdl-season-30-europe',
 '/matches/2331282/avangar-vs-x-kom-lootbet-season-1',
 '/matches/2331201/valiance-vs-ldlc-united-masters-league',
 '/matches/2331275/big-frames-vs-gh

In [22]:
all_matches[0].split('/')[2]

'2331080'

In [23]:
#div = soup.find_all('div', class_=["padding", "preformatted-text"])[0]

In [24]:
#div.text.split('*')[0]

In [25]:
def GetResult(soup):
    t1_div = soup.find_all('div', class_ = "team1-gradient")
    result = "tie"
    if t1_div != []:
        if t1_div[0].find_all('div', class_ = ["won"]) != []:
            result = 1
        elif t1_div[0].find_all('div', class_ = ["lost"]) != []:
            result = 0
    return result

In [26]:
def GetBestOf(soup):
    div = soup.find_all('div', class_=["padding", "preformatted-text"])[0].text
    #print(div)
    bestOf = div.split('*')[0].replace('\n', '')
    bestOf = bestOf.replace(" (LAN)", '').replace(" (Online)", '').replace("\n", '').strip()
    return bestOf

In [28]:
# Get data for individual matches
data = []
for m_url in all_matches:
    #m_url = '/matches/2329696/vitality-vs-g2-wesg-2018-north-west-europe'
    my_url = "https://www.hltv.org" + m_url
    try:
        page = requests.get(my_url, headers=headers)
        page_html = page.text

        soup = BeautifulSoup(page_html,"html.parser")
        team1 = soup.find_all('div', class_ = ["teamName"])[0].text
        team2 = soup.find_all('div', class_ = ["teamName"])[1].text
        result = GetResult(soup)
        if result == "tie":
            continue
        matchId = m_url.split('/')[2]
        
        bestOf = GetBestOf(soup)
        h2h = soup.find_all('div', class_ = ["head-to-head"])[0].find_all('div', class_ = ["bold"])
        t1wins = h2h[0].text
        t2wins = h2h[2].text
        overtimes = h2h[1].text
        team1pastMatches = soup.find_all('table', class_=["table matches"])[0]
        team2pastMatches = soup.find_all('table', class_=["table matches"])[1]
        team1pastMatchesWon = len(team1pastMatches.find_all('td', class_=["won"]))
        team2pastMatchesWon = len(team2pastMatches.find_all('td', class_=["won"]))

        data_entry = [matchId, team1, team2, bestOf, t1wins, t2wins, overtimes, team1pastMatchesWon, team2pastMatchesWon, result]
        data.append(data_entry)
    except Exception as e:
        print(e)
        print(m_url)

list index out of range
/forums
list index out of range
/forums/threads/1999688/rigged-showmatch
list index out of range
/forums/threads/1999702/ence-wont-beat-astralis
list index out of range
/forums/threads/1999698/lul-furlan
list index out of range
/forums/threads/1999671/why-is-almost-every-finnish-pro-gay
list index out of range
/news/26214/isurus-full-send-qualify-for-epl-s9-americas
list index out of range
/forums/threads/1999699/im-feeling-thank-you
list index out of range
/forums/threads/1999700/polish-no-english
list index out of range
/forums/threads/1999680/poland-1-in-danger-zone
list index out of range
/forums/threads/1999668/i-have-fucked-pornstars
list index out of range
/forums/threads/1999240/can-you-be-fat-and-still-look-nice
list index out of range
/forums/threads/1999658/if-ence-wins
list index out of range
/forums/threads/1999652/all-fans-are-annoying
list index out of range
/forums/threads/1999657/sadokist-graffiti
list index out of range
/forums/threads/1999608/

In [76]:
h2h = soup.find_all('div', class_ = ["head-to-head"])[0].find_all('div', class_ = ["bold"])
t1wins = h2h[0].text
t2wins = h2h[2].text
overtimes = h2h[1].text

In [75]:
b

<div class="stats-menu-link bold">
<div class="all dynamic-map-name-short" id="all">All</div>
<div class="all dynamic-map-name-full" id="all">All maps</div>
</div>

In [81]:
team1pastMatches = soup.find_all('table', class_=["table matches"])[0]
team2pastMatches = soup.find_all('table', class_=["table matches"])[1]
team1pastMatchesWon = len(team1pastMatches.find_all('td', class_=["won"]))
team2pastMatchesWon = len(team2pastMatches.find_all('td', class_=["won"]))

In [29]:
columns = ["matchId", "team1", "team2", "bestOf", "t1wins", "t2wins", "overtimes", "team1pastMatchesWon", "team2pastMatchesWon", "result"]


In [30]:
import pandas as pd

In [31]:
df = pd.DataFrame(data=data, columns=columns)
df.to_csv("matches_results.csv")